In [ ]:
from glob import glob
import os
import pandas as pd
import numpy as np
import re
from PIL import Image
from random import randrange
import pydicom
import matplotlib.pyplot as plt

In [ ]:
#checnking the input files
#--- For Windows ---#
print(os.listdir("D:\\kaggle\\rsna"))
rootDir = "D:\\kaggle\\rsna"

#--- For Unix ---#
#print(os.listdir("/Users/Sanketh/Dropbox/Research/kaggle"))
#rootDir = "/users/Sanketh/Dropbox/Research/kaggle"

In [ ]:
#reading all dcm files into train and text
train = os.path.join(rootDir, "train")
train = sorted(glob(train+'\\*.dcm'))
#train = sorted(glob(train+'\\*.dcm'))

valid = os.path.join(rootDir, "test")
valid = os.listdir(valid)
#valid = sorted(glob(valid+'\\*.dcm'))

print("train files: ", train[0])
print("valid files: ", valid[0])

pd.reset_option('max_colwidth')

In [ ]:
train_df = pd.read_csv(os.path.join(rootDir,'stage_1_train.csv'))

In [ ]:
def window_image(img, window_center,window_width, intercept, slope):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    return img 

In [ ]:
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [ ]:
case = 0

data = pydicom.dcmread(train[case])

#print(data)
window_center , window_width, intercept, slope = get_windowing(data)


#displaying the image
img = pydicom.read_file(train[case]).pixel_array

img = window_image(img, window_center, window_width, intercept, slope)

In [ ]:
print(img.shape)

In [ ]:
plt.imshow(img, cmap='gray')